# Travail pratique 3.1 - Autoencodeurs avec Pytorch

Par **Nathan Painchaud** et **Pierre-Marc Jodoin** de l'Université de Sherbrooke, Canada.  Adapté par **Antoine Théberge**

Certaines cellules de ce notebook ne fonctionnent pas sur Google Colab. Vous devrez exécuter ce notebook sur une machine locale pour bien répondre aux questions.

Inspiré par un [tutoriel similaire](https://blog.keras.io/building-autoencoders-in-keras.html) pour Keras par François Chollet.

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
%%capture packages_install

# Make sure the repo's package and its dependencies are installed
!pip install -e .
!pip install -r requirements.txt

In [ ]:
%%capture project_path_setup

import sys

if '../' in sys.path:
    print(sys.path)
else:
    sys.path.append('../')
    print(sys.path)

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True) # Change according to your own Google Drive config
!cp -r drive/MyDrive/IFT780/tp3/* . # Change according to your own Google Drive config

## Téléchargeons les données

Si ce n'est pas déjà fait, téléchargez le fichier suivant:

https://drive.usercontent.google.com/download?id=1J6gjGM-TGfuWKRf8zkGVJPTOy7QCCCUQ&export=download&authuser=0

et décompressez-le.  Ensuite, copiez le nouveau répertoire "data/" là où se trouvent les fichiers ipynb.

## PyTorch

Pytorch est une des bibliothèques d'apprentissage profond les plus utilisées dans le monde. Par contre, malgré tous nos efforts afin de simplifier le code, certaines parties du travail pratique pourront sembler un peu cryptiques pour les personnes non-familières à Pytorch. Idéalement, les néofites devraient faire un ou deux tutoriels sur Pytorch avant de s'attaquer au travail pratique.

* [Deep Learning With PyTorch: A 60 Minute Blitz](https://pytorch.org/tutorials/beginner/deep_learning_60min_blitz.html)
* [Training a Classifier on CIFAR10](https://pytorch.org/tutorials/beginner/blitz/cifar10_tutorial.html)


### Commençons par visualiser nos données

Dans cette cellule, nous chargerons le jeu de données MNIST et visualiserons quelques-unes de ses images.

In [ ]:
import numpy as np
from torchvision.datasets import MNIST
from torchvision.transforms import transforms
from src.visualization.utils import display_data_samples

# MNIST consists of 28x28 images, so the size of the data is
data_shape = 28, 28
data_size = data_shape[0] * data_shape[1]

# Download and prepare data
transform = transforms.Compose([
    transforms.ToTensor(),
])
mnist_train = MNIST("./data", train=True, transform=transform)
mnist_test = MNIST("./data", train=False, transform=transform)

# Check data by displaying random images
samples_indices = np.random.randint(len(mnist_train), size=10)
mnist_img_list = [mnist_train[sample_idx][0] for sample_idx in samples_indices]
display_data_samples(data=mnist_img_list)

In [ ]:
# What are `mnist_train` and `mnist_test`?  Let's look at it.
print(mnist_train)
print(mnist_test)
print("ces 2 datasets sont des : ",type(mnist_test))

### Jeux de données PyTorch

Veuillez noter que `mnist_train` et `mnist_test` sont des "Pytorch `dataset`". Un `dataset`Pytorch encapsule les données sous la forme de tenseurs. Vous pouvez accéder aux données tel qu'indiqué dans la cellule suivante.

In [ ]:
import matplotlib.pyplot as plt

# Get the first training image and its class label
sample_image = mnist_train[0][0] # sample_image is a "PyTorch tensor"
sample_label = mnist_train[0][1] 

# Convert the Tensor into a numpy array
sample_image_np = sample_image.numpy()  
print("Image size = ", sample_image_np.shape)

# Call "squeeze" to remove the first dimension
sample_image_np = sample_image_np.squeeze(0)
print("Image size = ", sample_image_np.shape)

# Plot
plt.imshow(sample_image_np)
print("The image label is ", sample_label)

### Concevons un autoencodeur déterministe

Ici, nous concevrons un simple autoencodeur avec seulement des couches **denses** (pleinement connectées) et des fonctions d'activations **ReLUs**. Pytorch nomme ces couches **Linear**.

In [ ]:
from torch import nn

# Let's define the encoder architecture we want,
# with some options to configure the input and output size
def make_encoder(data_size, latent_space_size):
    return nn.Sequential(
        nn.Linear(data_size, 128),
        nn.ReLU(),
        nn.Linear(128, 64),
        nn.ReLU(),
        nn.Linear(64, latent_space_size),
    )

# Same thing for the decoder
def make_decoder(data_size, latent_space_size):
    return nn.Sequential(
        nn.Linear(latent_space_size, 64),
        nn.ReLU(),
        nn.Linear(64, 128),
        nn.ReLU(),
        nn.Linear(128, data_size),
        nn.Sigmoid(),
    )

# Now let's build our networks, with an arbitrary dimensionality of the latent space
# and an input and output size depending on the data.
encoder = make_encoder(data_size, 32)
decoder = make_decoder(data_size, 32)

## Questions:
    
* Quelle est la relation entre l'architecture de l'encodeur et du décodeur ?
* Pourquoi la fonction d'activation en sortie est-elle une **sigmoïde** plutôt qu'une autre fonction comme **ReLU** ou **tanh**?
* Quelle est la taille de l'espace latent du modèle ?

Définissons notre fonction de **propagation avant** (*forward pass*):

In [ ]:
import torch
import torch.nn.functional as F

def autoencoder_forward_pass(encoder, decoder, x):
    """AE forward pass.

    Args:
        encoder: neural net that predicts a latent vector
        decoder: neural net that projects a point in the latent space back into the image space
        x: batch of N MNIST images

    Returns:
        loss: crossentropy loss
        x_hat: batch of N reconstructed images
    """
    in_shape = x.shape  # Save the input shape
    encoder_input = torch.flatten(x, start_dim=1)   # Flatten the 2D image to a 1D tensor (for the linear layer)
    z = encoder(encoder_input)  # Forward pass on the encoder (to get the latent space vector)
    x_hat = decoder(z)  # Forward pass on the decoder (to get the reconstructed input)
    x_hat = x_hat.reshape(in_shape)    # Restore the output to the original shape
    loss = F.binary_cross_entropy(x_hat, x) # Compute the reconstruction loss
    return loss, x_hat

## Boucle d'entraînement

Pour entraîner notre modèle, nous aurons besoin d'un boucle d'entraînement.

In [ ]:
import os
from torch.utils.data import DataLoader
from tqdm.auto import tqdm

# Define some training hyperparameters
epochs = 25
batch_size = 256

def train(forward_pass_fn, encoder, decoder, optimizer, train_data, val_data, device="cuda"):
    # Create dataloaders from the data
    # Those are PyTorch's abstraction to help iterate over the data
    data_loader_kwargs = {"batch_size": batch_size, "num_workers": os.cpu_count() - 1, "pin_memory": True}
    train_dataloader = DataLoader(train_data, shuffle=True, **data_loader_kwargs)
    val_dataloader = DataLoader(val_data, **data_loader_kwargs)

    # Ensure that the networks are on the requested device (typically a GPU)
    encoder = encoder.to(device)
    decoder = decoder.to(device)

    fit_pbar = tqdm(range(epochs), desc="Training", unit="epoch")
    pbar_metrics = {"train_loss": None, "val_loss": None}
    for epoch in fit_pbar:

        # Train once over all the training data
        for x, _ in train_dataloader:
            x = x.to(device)    # Move the data tensor to the device
            optimizer.zero_grad()   # Make sure gradients are reset
            train_loss, _ = forward_pass_fn(encoder, decoder, x)    # Forward pass
            train_loss.backward()   # Backward pass
            optimizer.step()    # Update parameters w.r.t. optimizer and gradients
            pbar_metrics["train_loss"] = train_loss.item()
            fit_pbar.set_postfix(pbar_metrics)

        # At the end of the epoch, check performance against the validation data
        for x, _ in val_dataloader:
            x = x.to(device)    # Move the data tensor to the device
            val_loss, _ = forward_pass_fn(encoder, decoder, x)
            pbar_metrics["val_loss"] = val_loss.item()
            fit_pbar.set_postfix(pbar_metrics)

## Questions:

La fonction `train(...)` contient une **boucle typique d'entraînement Pytorch**. Celle-ci inclut une **propagation avant** (*forward pass*), une **propagation arrière** (*backward pass*), un **pas de gradient** (*gradient step*, `optimizer.step()`) et une **vérification de validation** (*validation check*). 

* Quel est l'utilité du `dataloader` ?
* Que fait la propagation avant considérant qu'elle utilise 2 réseaux de neurones? Quelles sont les entrées et sorties de cette fonction ? 


Enfin, entraînons !

In [ ]:
optimizer = torch.optim.Adam([*encoder.parameters(), *decoder.parameters()])
train(autoencoder_forward_pass, encoder, decoder, optimizer, mnist_train, mnist_test)

Regardons les résultats (images reconstruites) sur l'ensemble de test.

In [ ]:
from src.visualization.utils import display_autoencoder_results

display_autoencoder_results(mnist_test, lambda x: autoencoder_forward_pass(encoder, decoder, x.cuda())[1])

### Espace latent

Avant de poursuivre, retournez au début du notebook et réduisez la taille de l'espace latent à 2. Puis, réentraînez.

## Question :

* Pourquoi pensez-vous que l'espace latent de deux dimensions donne des reconstructions plus floues et moins précises ?

Ensuite, exécutez la cellule suivant pour visualiser l'espace latent.

In [ ]:
from src.visualization.latent_space import plot_latent_space

plot_latent_space(
    mnist_test,
    lambda x: encoder(torch.flatten(x, start_dim=1))
)

### Rendons notre autoencodeur variationnel

Les autoencodeurs variationnels (*variational autoencoders*, VAE) sont très similaires aux autoencodeurs. Trois différences les distinguent: 

* La sortie de l'encodeur du VAE produit des vecteurs de moyennes et variances.
* L'entrée du décodeur est un vecteur, échantilloné aléatoirement, d'une distribution Normale déterminée par les vecteurs en sortie de l'encodeur
* La fonction de perte a deux termes: la perte de reconstruction et la divergence de Kullback-Leibler.

In [ ]:
# This time, we start right away with a 2D latent space to visualize it easily afterwards
latent_space_size = 2

vae_encoder = make_encoder(data_size, latent_space_size * 2) 
vae_decoder = make_decoder(data_size, latent_space_size)

## Questions:

* La cellule précédente multiplie par 2 la taille de l'espace latent lors de l'instanciation de l'encodeur. Pourquoi ?
* Dans la prochaine cellule, une certaine opération, une astuce, est effectuée afin d'échantilloner l'espace latent. Quelle est cette astuce et pourquoi est-elle utilisée ?
* Quelle est la taille de notre espace latent ?

Redéfinissons notre fonction de propagation avant:

In [ ]:
def kl_div(mu, logvar):
    kl_div_by_samples = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp(), dim=1)
    return torch.mean(kl_div_by_samples)

def vae_forward_pass(encoder, decoder, x):
    """VAE forward pass.

    Args:
        encoder: neural net that predicts a mean and a logvar vector
        decoder: neural net that projects a point in the latent space back into the image space
        x: batch of N MNIST images

    Returns:
        loss: crossentropy + kl_divergence loss
        x_hat: batch of N reconstructed images
    """
    in_shape = x.shape  # Save the input shape
    encoder_input = torch.flatten(x, start_dim=1)  # Flatten the 2D image to a 1D tensor (for the linear layer)
    encoding_distr = encoder(encoder_input)  # Forward pass on the encoder (to get the latent space posterior)
    # Nothing changed so far!

    # Second part of our trick!
    # We separate the (unique) latent space posterior into its two halves: mean and logvar
    mu, logvar = encoding_distr[:, :latent_space_size], encoding_distr[:, latent_space_size:]

    std = torch.exp(0.5 * logvar)
    eps = torch.randn_like(std)
    z = mu + eps * std

    # Decoding mostly stays the same. The only difference is the added 4th line below
    x_hat = decoder(z)  # Forward pass on the decoder (to get the reconstructed input)
    x_hat = x_hat.reshape(in_shape)    # Restore the output to the original shape
    loss = F.binary_cross_entropy(x_hat, x) # Compute the reconstruction loss
    loss += 5e-3 * kl_div(mu, logvar)  # Loss now also includes the KL divergence term
    return loss, x_hat

Entraînons notre autoencodeur variationnel !

In [ ]:
optimizer = torch.optim.Adam([*vae_encoder.parameters(), *vae_decoder.parameters()])
train(vae_forward_pass, vae_encoder, vae_decoder, optimizer, mnist_train, mnist_test)

Regardons les résultats sur l'ensemble de test

In [ ]:
display_autoencoder_results(mnist_test, lambda x: vae_forward_pass(vae_encoder, vae_decoder, x.cuda())[1])

## Plus de visualisation

Visualisons l'espace latent de notre autoencodeur, cette fois-ci variationnel.

In [ ]:
from src.visualization.latent_space import plot_latent_space

plot_latent_space(
    mnist_test,
    lambda x: vae_encoder(torch.flatten(x, start_dim=1))[:, :latent_space_size]
)

## Question:

* Quelle différence remarquez vous entre les deux espaces latents ?

Pour s'amuser avec l'espace latent, explorons-le. Changez la valeur de la variable `z` et ré-exécutez la cellule suivante afin de visualiser l'image reconstruite.

In [ ]:
import matplotlib.pyplot as plt 

z = [2, 0]  # 2D latent vector
 
z_torch = torch.tensor(z, dtype=torch.float).cuda()  # convert Z into a PyTorch tensor

sample = vae_decoder(z_torch).reshape(data_shape)  # decode the latent vector with the VAE decoder

plt.imshow(sample.detach().cpu().numpy()) # plot the resulting image